<a href="https://colab.research.google.com/github/Pallavi-Kolambkar/NEU_project/blob/master/BEL_to_SCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install http://download.pytorch.org/whl/cpu/torch-0.4.0-cp27-cp27mu-linux_x86_64.whl 
!pip install pyro-ppl

ERROR: torch-0.4.0-cp27-cp27mu-linux_x86_64.whl is not a supported wheel on this platform.
     |████████████████████████████████| 522kB 2.8MB/s 


In [0]:
#checking if the comment works
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint
import networkx as nx
# from pybel.examples import sialic_acid_graph as sag
# import pybel as pb
import json
import time
import csv
# from pybel.io.jupyter import to_jupyter
import torch
import pyro
import pandas as pd
import numpy as np
pyro.set_rng_seed(101)

In [0]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [0]:
from google.colab import files 
uploaded = files.upload() 
f = open('COVID19.jgf')
bel_statements = json.load(f)

Saving COVID19.jgf to COVID19.jgf


In [0]:
# type has the structure of (type_parent, type_children, type_relation):[[edges]]
label_dict = {'Abundance':['a', 'r', 'm', 'g', 'p','pop', 'composite', 'complex','frag','fus','loc','pmod','var'],
             'Process': ['bp', 'path','act'],
             'Transformation':['sec','surf','deg','rxn','tloc','fromLoc','products','reactants','toLoc']}
def get_information(jgf_file):
    # causal_relations = ['increases', 'decreases', 'directlyIncreases', 'directlyDecreases']
    types = []
    # nodes = set()
    parents = []
    parent_types = []
    children_types = []
    parent_labels = []
    children_labels = []
    children = []
    str_list = []
    df = pd.DataFrame()
    
    for edges in jgf_file['graph']['edges']:
        parent = edges['source']
        child = edges['target']
        relation = edges['relation']
        text = edges['label']
        if relation.find('crease') > 0:
            # nodes.add(parent)
            # nodes.add(child)
            parents.append(parent)
            children.append(child)
            str_list.append(parent + '*' + relation + '*' + child)   
            parent_type = parent[:parent.find('(')]
            children_type = child[:child.find('(')]
            types.append(relation)
            children_types.append(children_type)
            parent_types.append(parent_type)
            for label in label_dict:
                if parent_type in label_dict[label]:
                    parent_labels.append(label)
                elif parent_type == '': 
                    parent_labels.append('Others')
            for label in label_dict:
                if children_type in label_dict[label]:     
                    children_labels.append(label)
                elif children_type == '': 
                    children_labels.append('Others')
                    break
            # types.setdefault((parent_type,chidren_type,relation), []).append(parent + '*' + relation + '*' + child)
    df['parents'] = parents
    df['children'] = children
    df['types'] = types
    df['statements'] = str_list
    df['parent_types'] = parent_types
    df['parent_labels'] = parent_labels
    df['children_types'] = children_types
    df['children_labels'] = children_labels
    
    return df

In [0]:
df = get_information(bel_statements)
df = df[~df['children_labels'].isin(['Others'])] # filter the two with nested structures

In [0]:
df.head()

In [0]:
## identifying clusters
new_df = df.groupby(['children', 'types', 'children_types','children_labels']).apply(lambda x: [list(x['parents']), list(x['parent_types']),list(x['parent_labels'])]).apply(pd.Series)

In [0]:
new_df = new_df.reset_index(drop = False)
new_df.columns = ['children', 'types', 'children_types','children_labels', 'parents', 'parents_types','parent_labels']

In [0]:
new_df

In [0]:
new_df.iloc[16]

children           p(HGNC:ANGPT2)                                                                    
types              decreases                                                                         
children_types     p                                                                                 
children_labels    Abundance                                                                         
parents            [p(HGNC:VWF), act(complex(SCOMP:"AMP Activated Protein Kinase Complex"), ma(kin))]
parents_types      [p, act]                                                                          
parent_labels      [Abundance, Process]                                                              
Name: 16, dtype: object

# For each row we need to build a SCM 

## Jeremy's guide for dealing with node types

Let's talk about Abundance -> Process first.   If you have multiple Abundance parents, then the function could simply be a weighted sum of abundances followed by an activation function to determine whether the process is on or off.  In the simple case where you have two abundance parents, the Parent-Child function resembles an OR gate if the weights are large, because either abundance (or both) is sufficient to activate the process.  If the weights are small, then the the Parent-Child function resembles an AND gate, because you need both parents in high abundance.


7:20
Let's talk about  Process -> Transformation next.    In this case, if the process is active, then the transformation is a continuous variable.  If the process in inactive, then the transformation cannot occur, so it is zero.


7:21
Let's talk about (Abundance*, Process) -> Transformation.  In this case, the transformation is a potentially nonlinear function of abundance if the Process is active, but zero otherwise.


7:23
if Transformation -> Abundance, then we have a continuous in, and continuous out, but it may be nonlinear.


7:24
Process -> Abundance implies  continuous if Process is active and 0 otherwise.


7:25
All of these rules are flipped if the causal relation is decreases instead of increases


7:26
If Abundance -> Transformation we also have (potentially nonlinear) continuous to continuous.


In [0]:
import pyro.distributions as dist
def get_distribution(node_type):
    if node_type == 'Abundance':
        return dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))
    if node_type == 'Process':
        return dist.Categorical(torch.tensor([0.5,0.5]))
    else:
        return dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))

In [0]:
def SCM(parent_label,child_label,relation,threshold,w):
  parents = []
  process = []
  abundance = []
  transformation = []
  for i in range(len(parent_label)):
    parent = pyro.sample("par_%s" % i, get_distribution(parent_label[i]))
    parents.append(parent)
    if parent_label[i] =='Process':
      process.append(parent)
    if parent_label[i] == 'Abundance':
      abundance.append(parent)
    if parent_label[i] == 'Transformation':
        transformation.append(parent)


  if child_label != 'Transformation':
    
    if relation == 'increases':
      ## process is more like an OR switch
      if sum(process) > 0 :
        child_N =  sum([x*y for x,y in zip(w ,abundance)]) + pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0)))
        # + sum([x*y*y for x,y in zip(w ,transformation)])
      else: 
        child_N = pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))) 
      if child_label == 'Process':
        check = lambda x : 1.0 if x > threshold else 0.0
        child_N = torch.tensor(check(child_N))
      child = pyro.sample('child',pyro.distributions.Delta(child_N))
      
    if relation == 'decreases':
      if sum(process) > 0:
        child_N = pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0)))
      else:
        child_N =  sum([x*y for x,y in zip(w ,abundance)]) + pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0)))
       # + sum([x*y*y for x,y in zip(w ,transformation)])
      if child_label == 'Process':
        check = lambda x : 0.0 if x > threshold else 1.0
        child_N = torch.tensor(check(child_N))
      child = pyro.sample('child',pyro.distributions.Delta(child_N))
    

  else:
    if relation == 'increases':
      ## process is more like an OR switch
      if sum(process) > 0 :
        child_N =  sum([x*y for x,y in zip(w ,abundance)]) + pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))) + \
        sum([x*y*y for x,y in zip(w ,transformation)])
      else: 
        child_N = pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))) 
      if child_label == 'Process':
        check = lambda x : 1.0 if x > threshold else 0.0
        child_N = torch.tensor(check(child_N))
      child = pyro.sample('child',pyro.distributions.Delta(child_N))
      
    if relation == 'decreases':
      if sum(process) > 0:
        child_N = pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))) 
      else:
        child_N =  sum([x*y for x,y in zip(w ,abundance)]) + pyro.sample("child_n", dist.LogNormal(torch.tensor(0.0),torch.tensor(1.0))) + \
        sum([x*y*y for x,y in zip(w ,transformation)])
      if child_label == 'Process':
        check = lambda x : 0.0 if x > threshold else 1.0
        child_N = torch.tensor(check(child_N))
      child = pyro.sample('child',pyro.distributions.Delta(child_N))

#     if relation == 'increases':
#       if sum(process) > 0:
#         child_N =  sum([x*y*y for x,y in zip(w ,parents)]) + pyro.sample("child_n", pyro.distributions.Normal(0, 1)) 
#       else: 
#         child_N = pyro.sample("child_n", pyro.distributions.Normal(0, 1)) 
#       child = pyro.sample('child',pyro.distributions.Delta(child_N))
#       # return{'parents': parents,'child': child}

#     if relation == 'decreases':
#       if sum(process) > 0:
#         child_N = pyro.sample("child_n", pyro.distributions.Normal(0, 1)) 
#       else:
#         child_N =  sum([x*y*y for x,y in zip(w ,parents)]) + pyro.sample("child_n", pyro.distributions.Normal(0, 1))
#       child = pyro.sample('child',pyro.distributions.Delta(child_N))
  #return child.numpy()
  return{'parents': parents,'child': child}

In [0]:
# get the model for each row
SCM_list = []
for i in range(new_df.shape[0]):
    if i in [2,3,4]:
      continue
    test = new_df.iloc[i]
    SCM_list.append(SCM(test['parent_labels'],test['children_labels'],test['types'],2,[3]*len(test['parent_labels'])))
SCM_list
len(SCM_list)

42

# Adding a KL Divergence test to check the correctness of distributions we get from SCM against the true distribution of our child nodes. 

In [0]:
# get the model for each row
#SCM_list23 = []
#SCM_list22 = []
#SCM_list33 = []
#true_children = []
#for i in range(new_df.shape[0]):
#    if i in [2,3,4]:
#        continue
#    test = new_df.iloc[i]
#    SCM_list22.append(SCM(test['parent_labels'],test['children_labels'],test['types'],2,[2]*len(test['parent_labels'])))
#    SCM_list23.append(SCM(test['parent_labels'],test['children_labels'],test['types'],2,[3]*len(test['parent_labels'])))
#    SCM_list33.append(SCM(test['parent_labels'],test['children_labels'],test['types'],3,[3]*len(test['parent_labels'])))
#    true_children.append(pyro.sample("child", get_distribution(test['children_labels'])))

In [0]:
import numpy as np
from scipy.stats import norm
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
def KL(P,Q):
    """ Epsilon is used here to avoid conditional code for
    checking that neither P nor Q is equal to 0. """
    epsilon = 0.00001

    # You may want to instead make copies to avoid changing the np arrays.
    P = P+epsilon
    Q = Q+epsilon

    divergence = np.sum(P*np.log(P/Q))
    return divergence

In [0]:
print(KL(np.asarray(SCM_list23[5]), np.asarray(true_children[5])))

-0.0001151293546492023


#### Linear regression to get weights from SCM


In [0]:
#import pandas as pd
#samples_df = pd.DataFrame(SCM_list22)
#samples_df.head()
#samples_df.tail()

,parents,child
37,[tensor(1.0003)],tensor(3.7504)
38,[tensor(0.4245)],tensor(2.8604)
39,[tensor(0.3862)],tensor(1.3235)
40,[tensor(1.8983)],tensor(-0.4505)
41,[tensor(1.8402)],tensor(-1.0768)


In [0]:
#parent_matrix = []
#child_matrix = []
#for i in range(len(samples_df)):
#    parent_row = [0] * len((max(new_df['parent_labels'], key=len)))
#    for j in range(len(samples_df['parents'][i])):
#        parent_row[j] = samples_df['parents'][i][j].numpy().item()
#    parent_matrix.append(parent_row)
#    child_matrix.append(samples_df['child'][i].numpy().item())

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
regressor = LinearRegression()  
#regressor.fit(parent_matrix, child_matrix) #training the algorithm

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
#To retrieve the intercept:
#print(regressor.intercept_)
#For retrieving the slope:
#print(regressor.coef_)

In [0]:
samples_2= []
test = new_df.iloc[16]
for i in range(100):
  samples_2.append((SCM(test['parent_labels'],test['children_labels'],test['types'],2,[2]*len(test['parent_labels']))))

In [0]:
samples_2_df.head()

,parents,child,parent1,parent2
0,"[tensor(0.3249), tensor(0)]",tensor(2.2323),tensor(0.3249),tensor(0)
1,"[tensor(6.4018), tensor(1)]",tensor(0.4974),tensor(6.4018),tensor(1)
2,"[tensor(2.5888), tensor(0)]",tensor(5.5890),tensor(2.5888),tensor(0)
3,"[tensor(0.7808), tensor(0)]",tensor(2.8100),tensor(0.7808),tensor(0)
4,"[tensor(0.8992), tensor(1)]",tensor(1.1178),tensor(0.8992),tensor(1)


In [0]:
samples_2_df=pd.DataFrame(samples_2)
samples_2_df[['parent1','parent2']] = pd.DataFrame(samples_2_df['parents'].values.tolist(), index= samples_2_df.index)

In [0]:
sample2_fit=regressor.fit(samples_2_df[['parent1','parent2']].values,samples_2_df['child'])
sample2_fit

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
#To retrieve the intercept:
print(sample2_fit.intercept_)
#For retrieving the slope:
print(sample2_fit.coef_)

2.754476079811798
[ 1.56884789 -3.67540574]


#Counterfactual function

In [0]:
# counterfactual function to get the distribution of target variable by sampling
def counterfactual(model,observed_variable, observed_val,exogenous_variable,cf_variable,cf_val,target_variable):
    target = []
    # get the conditional model
    conditions = {}
    for i in range(len(observed_variable)):
        conditions[observed_variable[i]] = torch.tensor(observed_val[i])
    conditioned_model = pyro.condition(model, data=conditions)
    # get the posterior and marginal distribution
    posterior = pyro.infer.Importance(conditioned_model, num_samples=1000).run()
    marginal = EmpiricalMarginal(posterior,exogenous_variable)
    # get the do model
    dos = {}
    for i in range(len(cf_variable)):
        dos[cf_variable[i]] = torch.tensor(cf_val[i])
    do_model = pyro.do(model, data=dos)
    # sample target variable under updated posterior and do model
    for i in range(1000):
        samples = marginal()
        exog = {}
        for i in range(len(exogenous_variable)):
            exog[exogenous_variable[i]] = torch.tensor(samples[i])
        cf_model = pyro.condition(do_model, data= exog)
        trace_handler = pyro.poutine.trace(cf_model)
        trace = trace_handler.get_trace()

        target.append(trace.nodes[target_variable]['value'].numpy().item())

    return target

## To do:
1. type is causal relationship
2. we need to group parent and children type according to CBN legend
3. Start building SCM using the prior distribution guide Jeremy gave us
4. Train a linear model to confirm if the threshold / distribution works on the sample generated from SCM
5. Automate this process and test it on different BEL models we have

## Future scope
1. Counterfactuals and interventions

# Looking into the new data

In [0]:
with open('COVID-19-new.json') as f:
    covid19kg_dict=json.load(f)

In [0]:
#adding the new file data

parents=[]
children=[]
edgetype=[]
parent_type=[]
children_type=[]

for eachedge in covid19kg_dict[0]['nanopub']['assertions']:
    parents.append(eachedge['subject'])
    children.append(eachedge['object'])
    edgetype.append(eachedge['relation'])
    p=eachedge['subject']
    c=eachedge['object']
    ptype = p[:p.find('(')]
    ctype = c[:c.find('(')]
    parent_type.append(ptype)
    children_type.append(ctype)


In [0]:
newcovid_df=pd.DataFrame(list(zip(parents,parent_type,edgetype,children,children_type)),columns=['Parent','Parent_type','Relation','Child','Child_type'])

In [0]:
newcovid_df

,Parent,Parent_type,Relation,Child,Child_type
0,"bp(GO:""epidermal growth factor receptor signaling pathway"")",bp,increases,"act(complex(GO:""NF-kappaB complex""))",act
1,act(p(HGNC:EGF)),act,increases,"bp(GO:""epidermal growth factor receptor signaling pathway"")",bp
2,act(p(HGNC:ADAM17)),act,increases,"bp(GO:""tumor necrosis factor-mediated signaling pathway"")",bp
3,act(p(HGNC:ADAM17)),act,increases,act(p(HGNC:EGF)),act
4,"bp(GO:""positive regulation of interleukin-6 production"")",bp,increases,"path(MESH:""Severe Acute Respiratory Syndrome"")",path
5,"act(complex(GO:""NF-kappaB complex""))",act,increases,"bp(GO:""positive regulation of interleukin-6 production"")",bp
6,"bp(GO:""pattern recognition receptor signaling pathway"")",bp,increases,"act(complex(GO:""NF-kappaB complex""))",act
7,"a(TAX:""Severe acute respiratory syndrome coronavirus 2"")",a,increases,"bp(GO:""pattern recognition receptor signaling pathway"")",bp
8,"act(complex(p(HGNC:IL6),p(HGNC:STAT3)))",act,increases,"bp(GO:""positive regulation of interleukin-6 production"")",bp
9,act(p(HGNC:IL6R)),act,increases,"act(complex(p(HGNC:IL6),p(HGNC:STAT3)))",act


In [0]:
newcovid_df['Relation'].unique()

array(['increases', 'directlyIncreases', 'directlyDecreases'],
      dtype=object)